# Example run: Transport iteratively all sources from target2_moa into source 2 and visualize result with use_pca = True for 50 epochs


(Still need to run to check if it is fine -> LRZ disk space too full atm)

In [ ]:
import torch
import scanpy as sc
import numpy as np
import random

from ot_cfm.data_utils import load_adata
from ot_cfm.model import create_ot_cfm_model, create_ot_cfm_optimizer
from ot_cfm.training import train_cfm, save_ot_cfm_model, load_ot_cfm_model
from ot_cfm.transport import transport, create_training_dataloader, create_dataloader

In [ ]:
# Set up device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# Load target2_moa dataset
target2_moa = load_adata("../../data/Tim_target2_wellres_featuresimputed_druginfoadded_pycytominer.h5ad")

In [ ]:
# Initialize OT-CFM components
ot_cfm_model = create_ot_cfm_model(adata=target2_moa, use_pca=True)
ot_cfm_optimizer = create_ot_cfm_optimizer(ot_cfm_model)

In [ ]:
# Train and transport iteratively
from ot_cfm.model import ExactOptimalTransportConditionalFlowMatcher
FM = ExactOptimalTransportConditionalFlowMatcher(sigma=0.1)
translated_sources = []

In [ ]:
# Train and save OT-CFM models for each source
for source in target2_moa.obs["Metadata_Source"].unique():
    if source == "source_2":
        continue

    source_data = target2_moa[target2_moa.obs["Metadata_Source"] == source].copy()
    dataloader = create_training_dataloader(target2_moa, source, "source_2", batch_size=64, use_pca=True)
    trained_ot_cfm_model = train_cfm(ot_cfm_model, ot_cfm_optimizer, FM, dataloader, epochs=200, plot_loss=True)
    save_ot_cfm_model(ot_cfm_model, ot_cfm_optimizer, f"./saved_models/ot_cfm_{source}_to_source_2.pt")

In [ ]:
# Load and perform transport
for source in target2_moa.obs["Metadata_Source"].unique():
    if source == "source_2":
        continue

    dataloader = create_dataloader(target2_moa, source, batch_size=64, use_pca=True)
    load_ot_cfm_model(ot_cfm_model, ot_cfm_optimizer, f"./saved_models/ot_cfm_{source}_to_source_2.pt")
    translated_source_adata = transport(dataloader, ot_cfm_model, x1_label=source, device=device)
    translated_sources.append(translated_source_adata)
    sc.pl.pca(translated_source_adata, color="Metadata_Source")

In [ ]:
# Combine all translated sources with source_2
all_sources_adata = sc.concat(translated_sources, axis=0)
source_2_data = target2_moa[target2_moa.obs["Metadata_Source"] == "source_2"].copy()
all_sources_adata = sc.concat([all_sources_adata, source_2_data], join="outer")

In [ ]:
# Plot combined PCA and save
sc.pl.pca(all_sources_adata, color="Metadata_Source")
all_sources_adata.write('./outputs/run_3_final_adata_target2_moa_reconstructed_all_sources_OT_CFM.h5ad')